In [1]:
# import the modules
from entity import Entity
from exception import EntityDeleteHasInventoryError
from sql import *
from uuid import uuid4

In [2]:
# coonect using duckdb and validate tables
import duckdb as dd

# Create an in-memory DuckDB connection
con = dd.connect(':memory:')

# dd.sql("SELECT * FROM sqlite_scan('../data/inventory.db', 'inventory_sql');").show()
try:
    dd.sql("ATTACH '../data/inventory.db' AS inventory (TYPE sqlite);")
except:
    print("already attached")
dd.sql("USE inventory")

In [3]:
# define database queries
sql_inventory = '''
        select INV.hierarchy_level as lvl
            , INV.quantity as qty
            , CHI.description_singular as child
            , INV.position as pos
            , PAR.description_singular as parent
            , INV.inventory_id
            , INV.inventory_id_parent
            , PAR.entity_id as parent
            , CHI.entity_id as child
        from inventory_sql INV
        inner join entity_sql PAR on PAR.entity_id = INV.entity_id_parent
        inner join entity_sql CHI on CHI.entity_id = INV.entity_id_child
    '''

In [4]:
# try creating an item
item = Entity(description="a lovely plump piglet", id="1dd566b5-21ba-4f5c-ae90-c1841feab8b5")
print(item)
# return the database record
dd.sql("select * from entity_sql where entity_id = '1dd566b5-21ba-4f5c-ae90-c1841feab8b5'")

Lovely Plump Piglet (total inventory: 0) (1dd566b5-21ba-4f5c-ae90-c1841feab8b5)


┌──────────────────────────────────────┬──────────────────┬─────────────────────┬─────────┬──────────────┬──────────────────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│              entity_id               │ entity_id_parent │     entity_key      │  noun   │  adjectives  │ description_singular │  description_plural  │ base_hierarchy_level │    created_time     │
│               varchar                │     varchar      │       varchar       │ varchar │   varchar    │       varchar        │       varchar        │        int64         │      timestamp      │
├──────────────────────────────────────┼──────────────────┼─────────────────────┼─────────┼──────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ 1dd566b5-21ba-4f5c-ae90-c1841feab8b5 │ None             │ lovely_piglet_plump │ piglet  │ lovely plump │ lovely plump piglet  │ lovely plump piglets │                    0 │ 2025-03-07 08:36:29 │
└─────────

In [5]:
# update the entity and check the description in the database has updated
item2 = Entity("a plump lovely piglet")
piggy2 = select_entity(entity_key="lovely_piglet_plump")
# return the database record
dd.sql("select * from entity_sql where entity_id = '1dd566b5-21ba-4f5c-ae90-c1841feab8b5'")

┌──────────────────────────────────────┬──────────────────┬─────────────────────┬─────────┬──────────────┬──────────────────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│              entity_id               │ entity_id_parent │     entity_key      │  noun   │  adjectives  │ description_singular │  description_plural  │ base_hierarchy_level │    created_time     │
│               varchar                │     varchar      │       varchar       │ varchar │   varchar    │       varchar        │       varchar        │        int64         │      timestamp      │
├──────────────────────────────────────┼──────────────────┼─────────────────────┼─────────┼──────────────┼──────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┤
│ 1dd566b5-21ba-4f5c-ae90-c1841feab8b5 │ None             │ lovely_piglet_plump │ piglet  │ plump lovely │ plump lovely piglet  │ plump lovely piglets │                    0 │ 2025-03-07 08:36:29 │
└─────────

In [6]:
# add some inventory for the entity
insert_inventory(
    inventory_id = "6d652592-7301-4819-9660-3c5ea40f0b50",
    inventory_id_parent = "8ce8d635-c5c5-49d7-bdcf-2b5cfb2298fe", # lounge in vicarage
    entity_id_parent = "03dd5c43-d66e-4c3f-a6eb-eb67b47c0373", # lounge
    entity_id_child = "1dd566b5-21ba-4f5c-ae90-c1841feab8b5", # piglet
    quantity = 10,
)
print(item)
# check the updated inventory record
sql_where = "WHERE INV.inventory_id = '6d652592-7301-4819-9660-3c5ea40f0b50'"
sql_full = sql_inventory + sql_where
dd.sql(sql_full).show()

Lovely Plump Piglet (total inventory: 10) (1dd566b5-21ba-4f5c-ae90-c1841feab8b5)
┌───────┬───────┬─────────────────────┬─────────┬─────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┐
│  lvl  │  qty  │        child        │   pos   │ parent  │             inventory_id             │         inventory_id_parent          │                parent                │                child                 │
│ int64 │ int64 │       varchar       │ varchar │ varchar │               varchar                │               varchar                │               varchar                │               varchar                │
├───────┼───────┼─────────────────────┼─────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│     0 │    10 │ plump lovely piglet │ IN      │ loung

In [7]:
# delete the entities !!!Should fail with the EntityDeleteHasInventoryError!
print(item)
try:
    item.delete_existing()
except EntityDeleteHasInventoryError as e:
    print(e)       

Lovely Plump Piglet (total inventory: 10) (1dd566b5-21ba-4f5c-ae90-c1841feab8b5)
Cannot delete as entity has inventory


In [8]:
# remove the inventory so item can be deleted
delete_inventory("6d652592-7301-4819-9660-3c5ea40f0b50")
item.delete_existing()
piggy = select_entity(entity_key="lovely_piglet_plump")
print(item)
if piggy:
    print(dict(piggy))


Lovely Plump Piglet (total inventory: 0) (1dd566b5-21ba-4f5c-ae90-c1841feab8b5)


In [9]:
dd.close()